In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt


In [13]:
a = pd.read_csv('./data2/피오니 홍대점.csv')

In [14]:
a

,user_name,user_rank,num_response,user_star,time,rating,content
0,라멘처돌이,54,374,4.4,2024.04.17.,5.0,예전에는 특별했는데 우유빙수가 흔해지면서부터 굳이 갈 이유가 사라짐 오히려 무스케...
1,밍꾸,3,1,1.0,2024.04.15.,1.0,"(2월에 방문했습니다)10년전부터 유명한거 알고있었고,늘 사람많고 못가다가이번에 우..."
2,밥밥,16,34,4.1,2024.04.07.,5.0,딸케 맛있음 과하게 달지않고 크림도 안느끼하고 딱 맛잇엇어용
3,불꽃휴먼,11,24,4.0,2024.03.29.,4.0,맛이랑 분위기는 좋은데 비싸
4,banana lizard,25,79,3.2,2024.03.18.,4.0,"디저트는 그저 그렇고, 커피가 맛있어요!"
...,...,...,...,...,...,...,...
269,ak,5,158,3.9,2017.01.15.,5.0,NaN
270,나경,22,714,3.4,2016.07.05.,4.0,NaN
271,그리고,22,56,3.5,2016.03.21.,5.0,먹어본 딸기 케이크중 가장 맛있었음너무 달지않지도 느끼하지도 않음
272,전주희,39,446,3.2,2016.03.11.,5.0,봄이라 더 맛있다


In [ ]:
import base64

In [18]:
### url 가져오기

## 카카오 맵 
driver = webdriver.Chrome()
driver.get('https://map.kakao.com/')
time.sleep(2)

## input 창에 검색어 입력
searchbox = driver.find_element(By.ID, 'search.keyword.query')
searchbox.send_keys('홍대 맛집'+ Keys.ENTER)
time.sleep(3)

## 상세보기 주소 가져오기 
list_url = []

## 아래 1,2,3,4,5 리스트 번호 다 뜨도록
# 장소 더보기 클릭
driver.find_element(By.ID, 'info.search.place.more').click()
time.sleep(2)
# 1번리스트로 돌아오기
driver.find_element(By.ID, 'info.search.page.no1').click()
time.sleep(2)

for count in range(1):
    
    for inx in range(2, 6):
        
        # 현재 리스트의 상세보기 주소 스크래핑
        moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
        for i in range(len(moreview_element)):
            list_url.append(moreview_element[i].get_attribute('href'))
        # 다음 리스트 클릭
        driver.find_element(By.ID, f'info.search.page.no{inx}').click()
        # 5번 리스트 저장
        if inx == 5:
            moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
            for i in range(len(moreview_element)):
                list_url.append(moreview_element[i].get_attribute('href'))
        time.sleep(2)
    # 다음 리스트 페이지 넘어가기
    driver.find_element(By.ID ,'info.search.page.next').click()
    time.sleep(2)



## 웹페이지 종료
driver.close()

In [ ]:
import base64

## 정보 넣을 빈 딕셔너리
data = {
    'merchant' : [],
    'category' : [],
    'star' : [],
    'address' : [],
    'oper_time' : [],
    'starCount' : [],
    'reviewCount' : []
}
data2 = {
    'user_name' : [],
    'user_rank' : [],
    'num_response' : [],
    'user_star' : [], 
    'time' : [],
    'rating' : [],
    'content' : []
}



### 기본 정보 크롤링 (상세보기 get으로 열어서 가져오기)

for i in range(len(list_url)):

   
    ## 상세보기 웹페이지 열기
    driver = webdriver.Chrome()
    driver.get(list_url[i])
    driver.implicitly_wait(3)

    # 영업시간 더 보기 클릭
    try:
        oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')
        oper_more[0].click()
        time.sleep(2)  

    # 영업시간 더보기가 없는 경우
    except:
        pass

    # 후기 더보기 클릭
    try:
        link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
        while link_more.text == "후기 더보기":
            link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
            # 마지막 후기 더보기까지 눌렀으면 멈추기
            if link_more.text != "후기 접기":
                link_more.click()
                time.sleep(0.3)
    # 후기 더보기 없는 경우        
    except:
        pass
    time.sleep(0.1)
  
    ## 상세page bs4 파싱 분석
    soup = bs(driver.page_source, 'lxml')

    ##canvas 이미지 저장

    test_image = driver.execute_script("return document.querySelectorAll('canvas')[1].toDataURL();")
    image_data = test_image.split(',')[1]

    ## Base64 디코딩하여 이미지 데이터 추출
    image_data_decoded = base64.b64decode(image_data)

    test_image2 = driver.execute_script("return document.querySelectorAll('canvas')[2].toDataURL();")
    image_data2 = test_image2.split(',')[1]

    ## Base64 디코딩하여 이미지 데이터 추출
    image_data_decoded2 = base64.b64decode(image_data2)


        
    # 웹페이지 닫기
    driver.close()



    ## 매장명
    mer = soup.find('h2', {'class':'tit_location'}).get_text()
    data['merchant'].append(mer)
    ## 분류
    data['category'].append(soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip())
    ## 주소
    data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", " "))

    ## 별점, 리뷰 수 
    # 별점 있는 경우
    if len(soup.find_all('span',{'class':'color_b'})) > 1:
        data['star'].append(soup.find_all('span', {'class':'color_b'})[0].get_text().strip('점'))
        data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
        data['reviewCount'].append(soup.find_all('span', {'class':'color_b'})[1].get_text().strip('개'))
    # 별점 없는 경우
    else:
        data['star'].append('0')
        data['starCount'].append('0')
        data['reviewCount'].append(soup.find('span', {'class':'color_b'}).get_text().strip('개'))

    ## 영업시간
    # 더보기 있는 경우
    if oper_more:    
        data['oper_time'].append(soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip())
    # 더보기 없는 경우
    else:
        data['oper_time'].append(soup.find('ul', {'class':'list_operation'}).get_text().strip().replace("\n", ""))


    # 리뷰 목록 잡기
    reveiw_listed = soup.find_all('li', {'data-ismy':'false'})

    #### df2 만들기 (리뷰df)
    for j in range(len(reveiw_listed)):
        ## 유저 이름
        data2['user_name'].append(soup.find_all('span',{'class':'txt_username'})[j].get_text())
        ## 유저 레벨
        data2['user_rank'].append(soup.find_all('span', {'class' : 'txt_badge'})[j].get_text().strip('레벨'))
        ## 유저 후기수
        data2['num_response'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[0].get_text()
        )
        ## 유저 별점 평균
        data2['user_star'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[1].get_text()
        )
        ### 리뷰 작성 시간
        data2['time'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find('span',{'class':'time_write'}).get_text()
        )
        ## 리뷰 내용
        data2['content'].append(
            soup.find_all('p', {'class':'txt_comment'})[j].get_text().strip('더보기')
        )

        ## 유저가 평가한 별점 
        # 너비를 추출하여 별점 계산 (너비는 별점의 백분율을 나타냄)
        style_attribute = soup.find_all('div', {'class':'star_info'})[j].find('span',{'class':'ico_star inner_star'})['style']
        width_percent = float(style_attribute.split(':')[1].strip('%;'))
        rating_out_of_five = width_percent / 20  # 별점은 100%를 5로 나눈 값
        data2['rating'].append(rating_out_of_five)

    ## 데이터 프레임으로 만들기
    df2 = pd.DataFrame(data2)
    df2.to_csv(f'./data2/{mer}.csv',index = False)


    # 이미지 데이터를 파일로 저장
    with open(f'./image/{mer}1.png', 'wb') as f:
        f.write(image_data_decoded)

    with open(f'./image/{mer}2.png', 'wb') as f:
        f.write(image_data_decoded2)

    print(f"{i+1}번째 데이터 입력 완료, {mer}.csv 저장 완료, {mer}.png 저장 완료")

df = pd.DataFrame(data)
df.to_csv("./data1/data.csv", index=False)

In [19]:
import concurrent.futures
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd

# 함수 정의
def scrape_data(url):
    data = {'merchant': [], 'category': [], 'address': [], 'star': [], 'starCount': [], 'reviewCount': [], 'oper_time': []}
    data2 = {'user_name': [], 'user_rank': [], 'num_response': [], 'user_star': [], 'time': [], 'content': [], 'rating': []}

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(3)

    try:
        oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent="main_info,more_timeinfo"]')
        if oper_more:
            oper_more[0].click()
            time.sleep(2)
    except:
        pass

    # 후기 더보기 클릭
    try:
        link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
        while link_more.text == "후기 더보기":
            link_more.click()
            time.sleep(0.3)
    except:
        pass

    soup = bs(driver.page_source, 'html.parser')

    data['merchant'].append(soup.find('h2', {'class': 'tit_location'}).get_text())
    data['category'].append(soup.find('span', {'class': 'txt_location'}).get_text().split(':')[1].strip())
    data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", ""))
    if len(soup.find_all('span', {'class': 'color_b'})) > 1:
        data['star'].append(soup.find_all('span', {'class': 'color_b'})[0].get_text().strip('점'))
        data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
        data['reviewCount'].append(soup.find_all('span', {'class': 'color_b'})[1].get_text().strip('개'))
    else:
        data['star'].append('0')
        data['starCount'].append('0')
        data['reviewCount'].append(soup.find('span', {'class': 'color_b'}).get_text().strip('개'))
    if oper_more:
        data['oper_time'].append(soup.find_all('ul', {'class': 'list_operation'})[1].get_text().strip())
    else:
        data['oper_time'].append(soup.find('ul', {'class': 'list_operation'}).get_text().strip().replace("\n", ""))
    review_listed = soup.find_all('li', {'data-ismy': 'false'})

    for j in range(len(review_listed)):
        data2['user_name'].append(soup.find_all('span', {'class': 'txt_username'})[j].get_text())
        data2['user_rank'].append(soup.find_all('span', {'class': 'txt_badge'})[j].get_text().strip('레벨'))
        data2['num_response'].append(soup.find_all('div', {'class': 'unit_info'})[j].find_all('span', {'class': 'txt_desc'})[0].get_text())
        data2['user_star'].append(soup.find_all('div', {'class': 'unit_info'})[j].find_all('span', {'class': 'txt_desc'})[1].get_text())
        data2['time'].append(soup.find_all('div', {'class': 'unit_info'})[j].find('span', {'class': 'time_write'}).get_text())
        data2['content'].append(soup.find_all('p', {'class': 'txt_comment'})[j].get_text().strip('더보기'))
        style_attribute = soup.find_all('div', {'class': 'star_info'})[j].find('span', {'class': 'ico_star inner_star'})['style']
        width_percent = float(style_attribute.split(':')[1].strip('%;'))
        rating_out_of_five = width_percent / 20
        data2['rating'].append(rating_out_of_five)

    driver.close()

    df2 = pd.DataFrame(data2)
    df2.to_csv(f'./data2/{soup.find("h2", {"class": "tit_location"}).get_text()}.csv', index=False)

    print(f"{url} 데이터 입력 완료, {soup.find('h2', {'class':'tit_location'}).get_text()}.csv 저장 완료")

    return data

# 메인 코드
url_a = [list_url]  # 매장 URL 리스트

data_list = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    for result in executor.map(scrape_data, url_a):
        data_list.append(result)

df = pd.DataFrame(data_list)
df.to_csv("./data1/data.csv", index=False)

InvalidArgumentException: Message: invalid argument: 'url' must be a string
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00007FF7956C1522+60802]
	(No symbol) [0x00007FF79563AC22]
	(No symbol) [0x00007FF7954F7CE4]
	(No symbol) [0x00007FF79558AC3E]
	(No symbol) [0x00007FF79556AB7A]
	(No symbol) [0x00007FF79558A224]
	(No symbol) [0x00007FF79556A923]
	(No symbol) [0x00007FF795538FEC]
	(No symbol) [0x00007FF795539C21]
	GetHandleVerifier [0x00007FF7959C413D+3217821]
	GetHandleVerifier [0x00007FF795A060D7+3488055]
	GetHandleVerifier [0x00007FF7959FF05F+3459263]
	GetHandleVerifier [0x00007FF79577B866+823494]
	(No symbol) [0x00007FF795645FBF]
	(No symbol) [0x00007FF795640EE4]
	(No symbol) [0x00007FF795641072]
	(No symbol) [0x00007FF7956318C4]
	BaseThreadInitThunk [0x00007FF9F2537344+20]
	RtlUserThreadStart [0x00007FF9F41E26B1+33]
